In [2]:
!pip install strands-agents[mistral] python-dotenv

  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
  Using cached strands_agents-1.14.0-py3-none-any.whl.metadata (13 kB)
  Using cached docstring_parser-0.17.0-py3-none-any.whl.metadata (3.5 kB)
  Using cached mcp-1.20.0-py3-none-any.whl.metadata (85 kB)
  Using cached opentelemetry_api-1.38.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_instrumentation_threading-0.59b0-py3-none-any.whl.metadata (2.1 kB)
  Using cached opentelemetry_sdk-1.38.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mistralai-1.9.11-py3-none-any.whl.metadata (39 kB)
  Using cached httpx_sse-0.4.3-py3-none-any.whl.metadata (9.7 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydantic_settings-2.11.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached python_multipart-0.0.20-py3-none-any.whl.metadata (1.8 kB)
  Using cached sse_starlette-3.0.3-py3-none-any.whl.metadata (12 kB)
  Using cached opentelemetry_instrumentation-0.59b0-py3-none-any.w

In [1]:
import os
import sys
import dotenv
import json
from datetime import datetime

from pathlib import Path
from typing import List

sys.path.append('..')

from src.utils import (
    save_json,
    read_json,
    load_file_content,
    get_job_paths,
    get_training_paths,
    sanity_check,
    chat_with_persona,
    validate_submission_format,
    track_api_call,  # Cost tracking from utils
    print_cost_summary,  # Cost summary from utils
    reset_cost_tracker  # Reset cost tracker from utils
)

from live_chat.agents.agent_utils import matching_agent_process_jobs
from live_chat.models.persona_info import(
    InterviewState,
    UserInterview
)

from live_chat.models.interview_info import(
    InterviewInfo
)

from live_chat.prompts.interview_prompt import(
    INITIAL_INTERVIEW_PROMPT_REVERSE,
    JOB_EXTENSION_INTERVIEW_PROMPT
)

from live_chat.my_utils import (
    save_json,
    read_json
)
from live_chat.agents.conversation_agent_functions import (
    route_message_to_agent
)
from live_chat.agents.parsing_functions import (
    parse_conversation,
    parse_conversation_for_job_persona_info
)

# Load API key from .env file
dotenv.load_dotenv("../env")

# Check if we're good to go
if not os.getenv("MISTRAL_API_KEY"):
    print("❌ No MISTRAL_API_KEY found!")
    print("Create an env file with your API key")
else:
    print("✅ API key found, we're ready to roll")

✅ API key found, we're ready to roll


In [2]:
CONVERSATIONS_DIR = Path('../live_chat/conversations')
DATA_DIR = Path('../live_chat/data')

In [4]:
userInterview = UserInterview()

interview = InterviewInfo()

timestamp = datetime.now().strftime('%Y_%m_%d__%H_%M_%S')

conversation_id = timestamp

filename = f"user_interview_{conversation_id}.json"
user_interview_path = CONVERSATIONS_DIR / filename
if not user_interview_path.exists():
    userInterview_data = userInterview.model_dump_json()
    save_json(user_interview_path, userInterview_data)
else:
    userInterview_data = json.loads(read_json(user_interview_path))
    userInterview = UserInterview.model_validate(userInterview_data)

counter = 0

while True and counter < 30:
    counter = counter + 1
    user_input = input("Type something (or 'exit' to quit): ")

    if user_input.lower() == 'exit':
        print("Goodbye!")
        break
    else:
        userInterview.interview.append(f"User: {user_input}")

        #print(f"interview_state : {userInterview.interview_state}")

        agent_message_sent = False
        while agent_message_sent is False:
            #print('---')
            #print('PROCESS AGENT RESPONSE')
            agent_response = route_message_to_agent(userInterview)
            if agent_response is None:
                print('INTERVIEW STOPPED DUE TO NO MORE AGENT RESPONSE')
                break
            elif agent_response.conversation_finished is False:
                #print('PARSE AGENT RESPONSE')
                print(agent_response.message)
                agent_message_sent = True
                agent_message = agent_response.message
                userInterview.interview.append(f"Assistant: {agent_message}")
                # print(userInterview)
                userInterview_data = userInterview.model_dump_json()
                save_json(user_interview_path, userInterview_data)
            else:
                #print('PROCESS INTERVIEW STATE CHANGE')
                agent_message_sent = False
                #print('...')

                userInterview_data = userInterview.model_dump_json()
                userInterview = parse_conversation(userInterview)
                userInterview_data = userInterview.model_dump_json()
                save_json(user_interview_path, userInterview_data)

                #print(agent_response.conversation_finished)

                if userInterview.interview_state == InterviewState.STOP:
                    print('Thank you for your time. I have all the elements I need to process the analysis of your goals.')
                    break
                #agent_response = route_message_to_agent(userInterview)


Type something (or 'exit' to quit):  Hi


Hello! Welcome to our career assistance service. I'm here to help you get started. Are you interested in finding a job, finding training opportunities, or just getting some information?


Type something (or 'exit' to quit):  I want information about finance area


Great ! Do you have any specific questions about the finance area, or would you like general information to start with ?


Type something (or 'exit' to quit):  What jobs exists ?


There are many jobs in the finance area, such as financial analyst, accountant, financial advisor, and investment banker. Do you want to know more about these jobs, or are you interested in finding a job or training in this field ?


Type something (or 'exit' to quit):  What about financial analyst


A financial analyst is responsible for analyzing financial data, preparing reports, and making recommendations to management. They may also be involved in forecasting and budgeting. Is there anything specific you would like to know about this role, or would you like to explore other job options or training opportunities in the finance field?


KeyboardInterrupt: Interrupted by user